This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'jXp37fs_eyjC-XcakBy5'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules

import requests
import json

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2019-05-29&end_date=2019-05-30&api_key='+API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

json_dict = r.json()
json_dict

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2019-06-24T22:33:36.929Z',
  'newest_available_date': '2019-06-24',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2019-05-29',
  'end_date': '2019-05-30',
  'data': [['2019-05-30',
    None,
    85.9,
    84.0,
    85.85,
    None,
    117606.0,
    10048622.0,
    None,
    None,
    None],
   ['2019-05-29',
    None,
    87.7,
    83.65,
    84.1,
    None,
    2532

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

<h3> Task #1 Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).


In [5]:
afx_17 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY)

<h3> Task #2 Convert the returned JSON object into a Python dictionary.


In [6]:
# Load request text as json to dict. Verify.

afx17dict = json.loads(afx_17.text)

type(afx17dict)

dict

<h3> Task #3 Calculate what the highest and lowest opening prices were for the stock in this period.


In [7]:
# Extract column names for easy reference.

afx17dict['dataset']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [8]:
# Column [1]: Open
# Calculate Highest and Lowest Opening Prices for this Period

open_prices = [daily[1] for daily in afx17dict['dataset']['data'] if daily[1] is not None]
min_open = min(open_prices)
max_open = max(open_prices)

print('The highest opening price in 2017 was $' + str(max_open) + " per share.")
print('The lowest opening price in 2017 was $' + str(min_open) + " per share.")

The highest opening price in 2017 was $53.11 per share.
The lowest opening price in 2017 was $34.0 per share.


<h3> Task #4 What was the largest change in any one day (based on High and Low price)?

In [9]:
# Column [2]: High
# Column [3]: Low
# Get max of daily high minus daily low

daily_change = [daily[2]-daily[3] for daily in afx17dict['dataset']['data'] if daily[2:3] != None]
largest_change = round(max(daily_change), 2)

print('The largest change in any one day in 2017 was $' + str(largest_change) + " per share.")

The largest change in any one day in 2017 was $2.81 per share.


<h3> Task #5 What was the largest change between any two days (based on Closing Price)?

In [10]:
# Column [4]: Close
# Start with close of first day, no change on day one.

close_delta_max = 0
prev_close =  afx17dict["dataset"]["data"][0][4]

for row in afx17dict["dataset"]["data"][1:]:
    close = row[4]
    close_delta = abs(prev_close - close)
    prev_close = close
    if close_delta > close_delta_max:
        close_delta_max = close_delta

print('The largest change between any two days in 2017 was $' + str(round(close_delta_max,2)) + " per share.")

The largest change between any two days in 2017 was $2.56 per share.


<h3> Task #6 What was the average daily trading volume during this year?

In [11]:
# Column [6]: Traded volume

volume = [row[6] for row in afx17dict["dataset"]["data"]]
vol_avg = sum(volume) / len(volume)

print("The average daily traded volume in 2017 was " + str(round(vol_avg, 1)) + " shares.")

The average daily traded volume in 2017 was 89124.3 shares.


<h3> Task #7 (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [12]:
# In this case volume has an odd number of values, but in case it didn't we have the else statement.

vol_sorted = sorted(volume)

if len(volume) % 2 != 0:
    vol_median = vol_sorted[len(volume) // 2]
else:
    vol_median = (vol_sorted[len(volume) // 2 - 1] + vol_sorted[len(volume) // 2]) / 2

print("The median daily traded volume in 2017 was " + str((round(vol_median, 1))) + " shares")

The median daily traded volume in 2017 was 76286.0 shares
